<a href="https://colab.research.google.com/github/daycardoso/PredictCost/blob/main/PredictCostDecisionTreeRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho CMP263 - Aprendizagem de Máquina - INF/UFRGS

## Modelo 1 - Decision tree sem poda

As árvores de decisão são conhecidas por possuírem um baixo viés, ao mesmo tempo em que apresentam alta variância.
Isto é, o método é capaz de modelar fronteiras de decisão bastante complexas, o que, por um lado, é positivo, mas torna o algoritmo bastante suscetível a ruído ou a padrões nos dados de treino que não generalizam para instâncias de teste.
Por isso, técnicas de poda são fundamentais para o uso efetivo do modelo em dados novos.



### Objetivos da Atividade
* Analisar os impactos da característica de **variância** nas árvores de decisão.
* Analisar o efeito da **poda** em árvores de decisão.


## Carregamento dos Dados


### Obtenção e análise dos dados
O código abaixo carrega o dataset do kaggle e mostra algumas informações básicas sobre os dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
# import glob

# arquivos = glob.glob('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/*.csv')



In [ ]:
# dfs = [pd.read_csv(f) for f in arquivos]
# df_unificado = pd.concat(dfs, axis=0, ignore_index=True)
# df_unificado.head()

In [ ]:
# Excluir a coluna type
# df_unificado = df_unificado.drop('type', axis=1)
# df_unificado.head()

In [ ]:
# # Garantir que não a duplicata de instancias evitando sobreposição entre os dados de treinamento e teste
# df_unificado = df_unificado.drop_duplicates().reset_index(drop=True)
# df_unificado.head(-50)

In [ ]:
# Salvar o dataset completo, sem duplicatas
# df_unificado.to_csv('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/df_unificado.csv', index=False)

In [ ]:
# Carregar o datset unificado
df_unificado = pd.read_csv('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/df_unificado.csv')

In [ ]:
# matriz contendo os atributos
X = df_unificado.iloc[:, :-1].values

# vetor contendo o custo, ou seja, a ultima coluna
y = df_unificado.iloc[:, -1].values

# nome de cada atributo
feature_names = df_unificado.columns[:-1]

# nome de cada classe
target_names = df_unificado.columns[-1]

print(f"Dimensões de X: {X.shape}\n")
print(f"Dimensões de y: {y.shape}\n")
print(f"Nomes dos atributos: {feature_names}\n")
print(f"Nomes das classes: {target_names}")

In [ ]:
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
import joblib

# 1) Cria um hold-out antes de qualquer CV
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

In [ ]:
pipeline = Pipeline([
    ('regressor', DecisionTreeRegressor(random_state=42))
])


In [ ]:
from sklearn.model_selection import RepeatedKFold

# 5×5 CV repetida: balanceia viés x variância na estimação
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)


In [ ]:
from sklearn.model_selection import cross_validate

scoring = {
    'R2': 'r2',
    'MSE': 'neg_mean_squared_error',
    'MAE': 'neg_mean_absolute_error',
    'MAPE': 'neg_mean_absolute_percentage_error',
    'MedAE': 'neg_median_absolute_error',
    'MaxE': 'max_error',
    'EVS': 'explained_variance',
}

cv_results = cross_validate(
    pipeline, X_train_full, y_train_full,
    cv=cv, scoring=scoring, return_train_score=True, n_jobs=-1
)

In [ ]:
# # Treinar o modelo
# # pipeline.fit(X_train_full, y_train_full)
# modelo = pipeline.fit(X_train_full, y_train_full)

In [ ]:
# 5) Gera predições no hold-out
# y_pred = pipeline.predict(X_test)

In [ ]:
# # 6) Salva TUDO num dict
# full_results = {
#     'pipeline': pipeline,
#     'X_test':   X_test,
#     'y_test':   y_test,
#     'y_pred':   y_pred,
#     'cv_results': cv_results,
#     'feature_names': feature_names
# }
# joblib.dump(full_results, '/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/decision_tree_regressor_full_results.pkl')

In [ ]:
# import joblib
# # Carregar o modelo
# pipeline = joblib.load('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/modelo_joblib.pkl')


In [ ]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    median_absolute_error,
    max_error,
    explained_variance_score
)
from IPython.display import display

# Carregar resultados
res = joblib.load('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/decision_tree_regressor_full_results.pkl')
model         = res['pipeline']
X_test        = res['X_test']
y_true        = res['y_test']
y_pred        = res.get('y_pred', model.predict(X_test))
feature_names = res.get('feature_names', getattr(X_test, 'columns', None))

# 1) Exibir métricas em tabela
metrics = {
    'R² Score'           : r2_score(y_true, y_pred),
    'MSE'                 : mean_squared_error(y_true, y_pred),
    'MAE'                 : mean_absolute_error(y_true, y_pred),
    'MAPE'                : mean_absolute_percentage_error(y_true, y_pred),
    'Median AE'           : median_absolute_error(y_true, y_pred),
    'Max Error'           : max_error(y_true, y_pred),
    'Explained Variance'  : explained_variance_score(y_true, y_pred)
}
df_metrics = pd.DataFrame.from_dict(metrics, orient='index', columns=['Valor']).round(4)
display(df_metrics)

# 2) Scatter Real vs Predito
plt.figure()
plt.scatter(y_true, y_pred)
min_val, max_val = np.min([y_true.min(), y_pred.min()]), np.max([y_true.max(), y_pred.max()])
plt.plot([min_val, max_val], [min_val, max_val])
plt.xlabel('Valor Real')
plt.ylabel('Valor Predito')
plt.title('Real vs Predito')
plt.show()

# 3) Histograma de resíduos
residuals = y_true - y_pred
plt.figure()
plt.hist(residuals, bins=50)
plt.xlabel('Resíduo (Real - Predito)')
plt.title('Histograma de Resíduos')
plt.show()

# 4) Boxplot de resíduos por quartil
df_r = pd.DataFrame({'Real': y_true, 'Resíduo': residuals})
df_r['Quartil'] = pd.qcut(df_r['Real'], 4, labels=[1, 2, 3, 4])
groups = [df_r[df_r['Quartil'] == q]['Resíduo'] for q in sorted(df_r['Quartil'].unique())]
plt.figure()
plt.boxplot(groups, labels=sorted(df_r['Quartil'].unique()))
plt.xlabel('Quartil de Valor Real')
plt.ylabel('Resíduo')
plt.title('Boxplot de Resíduos por Quartil')
plt.show()

# 5) Importância das features (se aplicável)
if hasattr(model, 'feature_importances_') and feature_names is not None:
    fi = pd.Series(model.feature_importances_, index=feature_names).sort_values(ascending=False)
    plt.figure(figsize=(8, 4))
    fi.plot(kind='bar')
    plt.ylabel('Importância')
    plt.title('Importância das Features')
    plt.tight_layout()
    plt.show()


###  poucas previsões com grandes desvios, possiveis outliers



In [ ]:
import numpy as np
import pandas as pd

# Suponha que você já tenha:
# y_true: valores reais
# y_pred: valores preditos
# X_test: features de teste (idealmente como DataFrame)

# Garante que y_test seja um array 1D
y_true_array = np.ravel(y_true)

# Cria DataFrame com todas as informações relevantes
df_erros = pd.DataFrame({
    'Valor Real': y_true_array,
    'Valor Predito': y_pred,
    'Erro Absoluto': np.abs(y_true_array - y_pred)
})


In [ ]:
# Ordena pelas maiores diferenças
df_maiores_erros = df_erros.sort_values(by='Erro Absoluto', ascending=False)

# Seleciona, por exemplo, os 10 piores casos
top_erros = df_maiores_erros.head(100)


In [ ]:
# Se X_test for DataFrame e tiver os mesmos índices
if not isinstance(X_test, pd.DataFrame):
    X_test_df = pd.DataFrame(X_test, columns=feature_names) # Use feature_names para nomes das colunas
else:
    X_test_df = X_test.copy() # Se já for DataFrame, apenas copie

X_test_df.reset_index(drop=True, inplace=True)
df_erros.reset_index(drop=True, inplace=True)

# Concatena features com os erros
df_completo = pd.concat([X_test_df, df_erros], axis=1)

# Pega os 10 piores casos com todas as features
top_casos_completos = df_completo.sort_values(by='Erro Absoluto', ascending=False).head(100)


In [ ]:
# Ver os X_test com maiores erros
top_casos_completos.head(100)

In [ ]:
import matplotlib.pyplot as plt

top_casos_completos[['Valor Real', 'Valor Predito']].plot(kind='bar', figsize=(12, 6))
plt.title('Comparação entre Valor Real e Valor Predito (Top 10 Maiores Erros)')
plt.xlabel('Casos com maior erro')
plt.ylabel('Valor')
plt.grid(True)
plt.show()


In [ ]:
import joblib
import matplotlib.pyplot as plt
from sklearn import tree

# Carrega o pipeline treinado
# res = joblib.load('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/full_results.pkl')
pipeline = res['pipeline']
feature_names = res.get('feature_names', None)

# Extrai o DecisionTreeRegressor
regressor = pipeline.named_steps['regressor']

# Desenha a árvore
plt.figure(figsize=(20, 10))
tree.plot_tree(
    regressor,
    feature_names=feature_names,
    filled=True,        # nós coloridos para melhor distinção
    rounded=True,       # cantos arredondados
    fontsize=10
)
plt.title("Visualização da Árvore de Decisão")
plt.tight_layout()
plt.show()


In [ ]:
# Se estiver usando Colab, pode ser necessário instalar o Graphviz no sistema:
# !apt-get install -qq graphviz
# !pip install -q graphviz

import joblib
import graphviz
from sklearn.tree import export_graphviz

# # 1) Carrega o pipeline treinado
# res = joblib.load('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/full_results.pkl')
# pipeline = res['pipeline']
# feature_names = res.get('feature_names', None)

# 2) Extrai o DecisionTreeRegressor
regressor = pipeline.named_steps['regressor']

# 3) Exporta para DOT
dot_data = export_graphviz(
    regressor,
    out_file=None,
    feature_names=feature_names,
    filled=True,
    rounded=True,
    special_characters=True
)

# 4) Renderiza com graphviz e exibe inline
graph = graphviz.Source(dot_data)
graph


## Variância nas Árvores de Decisão- EDITAR TUDO PARA REGREÇÃO





### Análise da Variação na Acurácia

A propriedade de variância também implica em efeitos na variabilidade do desempenho dos modelos. Para fins de exemplo, podemos usar a acurácia como medida de desempenho através das funções do scikit-learn. Entretanto, outras métricas de desempenho como Recall e Precisão, que são mais indicadas para problemas em que o número de instâncias por classe é desbalanceado (como é o caso deste conjunto de dados) poderiam também ser exploradas (a critério do aluno, podem ser adicionadas para observação, mas a questão deve ser respondida com base na acurácia).

O código abaixo executa repetidas vezes o treinamento das árvores de decisão, da mesma forma que no item *Analisando a Estrutura das Árvores*.
Modifique-o de forma a obter a acurácia para cada execução e então calcule a média, desvio padrão, máximo e mínimo dos valores. Use esses resultados para responder à **Questão 2**.

**Atenção: Não mude os valores que estão sendo passados para os parâmetros random_state para garantir a reprodutibilidade do código**.


### Análise de Instância individuais

1. Treine novamente uma árvore de decisão usando um novo conjunto de treino gerado com a função train_test_split. Utilize 20% de dados de teste e, desta vez, não **especifique valor nenhum para o random_state**.

2. Faça a predição para as instâncias especificadas abaixo e preencha na tabela do excel indicada no **Moodle** a classificação encontrada (0 para maligno e 1 para benigno).


## O Efeito da Poda

As árvores de decisão treinadas nos itens anteriores não possuíam nenhuma forma de poda. No entanto, é possível utilizar técnicas de poda através do scikit-learn. Como consequência, elas podem ter uma complexidade além do que é necessário na modelagem do problema.



### Exemplo de Pré-poda: profundidade máxima da árvore
Podemos especificar a profundidade máxima da árvore utilizando o parâmetro max_depth.

O código abaixo gera árvores de decisão com diferentes profundidades máximas e as avalia em termos de acurácia.

Observe que todas as árvores são treinadas e avaliadas com os mesmos conjuntos de treino e teste, visto que especificamos o parâmetro $random\_state = 0$.

Com base nesse código, e possíveis modificações que você faça a ele, responda à **Questão  4** do questionário.

**Não mude o valor que está sendo passado em random_state=0**.


### Exemplo de Pós-poda: Custo-complexidade

A biblioteca scikit-learn possui uma implementação de pós-poda por custo-complexidade, baseada no parâmetro de custo-complexidade $\alpha \ge 0$.

Na implementação descrita na biblioteca, é definido também um custo-complexidade efetivo do nodo. Quanto maior for a taxa de erros ao se podar a subárvore de um nodo, maior será seu custo-complexidade efetivo. Além disso, quanto maior for a complexidade (número de nodos terminais) da subárvore do nodo, menor será seu custo-complexidade efetivo.
Em resumo, um nodo com alto custo-complexidade efetivo é um nodo importante para diminuir a taxa de erros e com baixa complexidade.

Dentro da biblioteca, passamos um parâmetro $ccp\_alpha$ que serve como um custo-complexidade efetivo de corte: subárvores são podadas enquanto houver nodos com custo-complexidade menor do que o parâmetro $ccp\_alpha$.
Ou seja, quando maior for o parâmetro, mais intensa será a poda.

Para mais informações:
* https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning
* https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html

Use o código abaixo para resolver à **Questão 5**.